In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import os,sys
opj = os.path.join
from copy import deepcopy
import pickle as pkl

sys.path.append('../../src')
sys.path.append('../../src/dsets/cosmology')
from dset import get_dataloader
from viz import viz_im_r, cshow, viz_filters
from sim_cosmology import p, load_dataloader_and_pretrained_model
from losses import get_loss_f
from train import Trainer, Validator

# wt modules
from wavelet_transform import Wavelet_Transform, Attributer, get_2dfilts, initialize_filters
from utils import tuple_L1Loss, tuple_L2Loss, thresh_attrs, viz_list

In [90]:
import pywt
import warnings
from itertools import product

import numpy as np

from pywt._c99_config import _have_c99_complex
from pywt._extensions._dwt import idwt_single
from pywt._extensions._swt import swt_max_level, swt as _swt, swt_axis as _swt_axis
from pywt._extensions._pywt import Wavelet, Modes, _check_dtype
from pywt._multidim import idwt2, idwtn
from pywt._utils import _as_wavelet, _wavelets_per_axis

In [3]:
# get dataloader and model
(train_loader, test_loader), model = load_dataloader_and_pretrained_model(p, img_size=256)
torch.manual_seed(p.seed)
im = iter(test_loader).next()[0][0:1].numpy().squeeze()

In [93]:
db2 = pywt.Wavelet('db2')

In [105]:
start_level = 0
axes = (-2, -1)
trim_approx = False
norm = False

In [153]:
data = np.asarray(im)
# coefs = swtn(data, wavelet, level, start_level, axes, trim_approx, norm)
axes = [a + data.ndim if a < 0 else a for a in axes]
num_axes = len(axes)
print(axes)
wavelets = _wavelets_per_axis(db2, axes)
ret = []
level = 1

[0, 1]


In [164]:
i = 0
coeffs = [('', data)]
axis = axes[0]
wavelet = wavelets[0]
new_coeffs = []
subband, x = coeffs[0]
cA, cD = _swt_axis(x, wavelet, level=1, start_level=i,
                   axis=axis)[0]
new_coeffs.extend([(subband + 'a', cA),
                   (subband + 'd', cD)])
coeffs = new_coeffs

In [165]:
coeffs

[('a',
  array([[-1.05216005e-03,  4.46301093e-03, -1.56744849e-03, ...,
          -3.40364617e-03, -1.77054340e-03, -1.44116115e-03],
         [-3.58037930e-03, -2.80459132e-03, -5.55969309e-03, ...,
           1.54842995e-03,  2.39453069e-03,  5.39557356e-03],
         [ 6.62567886e-03,  1.75025803e-03,  3.65102151e-03, ...,
          -7.15488277e-04,  7.84642762e-05,  2.22519482e-03],
         ...,
         [ 5.51964529e-03,  3.12441518e-03,  2.25836551e-03, ...,
           2.26526055e-03, -5.27867861e-03, -1.13238748e-02],
         [ 5.45958895e-03,  6.21066662e-03,  4.41697240e-03, ...,
          -8.23037419e-03, -1.01424614e-02, -1.37235001e-02],
         [ 6.28706021e-03,  1.28138736e-02,  7.01082451e-03, ...,
          -1.27704535e-02, -1.10925213e-02, -1.23784877e-02]], dtype=float32)),
 ('d',
  array([[-4.5405813e-03, -3.1076092e-03, -4.4733770e-03, ...,
           1.7678754e-03,  1.2743192e-03,  3.2380112e-03],
         [-5.3215108e-04, -2.4327349e-03, -4.1219674e-04, ...,
 

In [92]:
def swtn(data, wavelet, level, start_level=0, axes=None, trim_approx=False,
         norm=False):

    wavelets = _wavelets_per_axis(wavelet, axes)
    if norm:
        if not np.all([wav.orthogonal for wav in wavelets]):
            warnings.warn(
                "norm=True, but the wavelets used are not orthogonal: \n"
                "\tThe conditions for energy preservation are not satisfied.")
        wavelets = [_rescale_wavelet_filterbank(wav, 1/np.sqrt(2))
                    for wav in wavelets]
    ret = []
    for i in range(start_level, start_level + level):
        coeffs = [('', data)]
        for axis, wavelet in zip(axes, wavelets):
            new_coeffs = []
            for subband, x in coeffs:
                cA, cD = _swt_axis(x, wavelet, level=1, start_level=i,
                                   axis=axis)[0]
                new_coeffs.extend([(subband + 'a', cA),
                                   (subband + 'd', cD)])
            coeffs = new_coeffs

        coeffs = dict(coeffs)
        ret.append(coeffs)

        # data for the next level is the approximation coeffs from this level
        data = coeffs['a' * num_axes]
        if trim_approx:
            coeffs.pop('a' * num_axes)
    if trim_approx:
        ret.append(data)
    ret.reverse()
    return ret

In [ ]:
def swt2(data, wavelet, level, start_level=0, axes=(-2, -1),
         trim_approx=False, norm=False):
    axes = tuple(axes)
    data = np.asarray(data)
    if len(axes) != 2:
        raise ValueError("Expected 2 axes")
    if len(axes) != len(set(axes)):
        raise ValueError("The axes passed to swt2 must be unique.")
    if data.ndim < len(np.unique(axes)):
        raise ValueError("Input array has fewer dimensions than the specified "
                         "axes")

    coefs = swtn(data, wavelet, level, start_level, axes, trim_approx, norm)
    ret = []
    if trim_approx:
        ret.append(coefs[0])
        coefs = coefs[1:]
    for c in coefs:
        if trim_approx:
            ret.append((c['da'], c['ad'], c['dd']))
        else:
            ret.append((c['aa'], (c['da'], c['ad'], c['dd'])))
    return ret